# Install for environment and set cuda

In [1]:
import torch
import os
!pip install pyyaml==5.1

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f "https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html"
# If there is not yet a detectron2 release that
# matches the given torch + CUDA version,
# you need to install a different pytorch.

# exit(0)  # After installation, you may need to
# "restart runtime" in Colab. This line can also restart runtime


torch:  1.9 ; cuda:  cu102
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html


# Import packages

In [2]:
import detectron2
import os
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2 import model_zoo
from detectron2.utils.visualizer import ColorMode
from detectron2.modeling import build_model
setup_logger()


# Set the training and testing path and register the coco instance

In [3]:
path_to_train_images = "./dataset/train/"
path_to_train_annotations = "annotations.json"
path_to_test_images = "./dataset/test/"
path_to_test_annotations = "./dataset/test_img_ids.json"
register_coco_instances("train", {}, path_to_train_annotations,
                        path_to_train_images)


# Set the model parameters and choose the pretrained model Mask RCNN ResNet 101

In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))  # 101
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 100000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[12/14 09:51:43 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[12/14 09:51:43 d2.data.datasets.coco]: Loaded 24 images in COCO format from annotations.json
[12/14 09:51:43 d2.data.build]: Removed 0 images with no usable annotations. 24 images left.
[12/14 09:51:43 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    cell    | 14598        |
|            |              |
[12/14 09:51:43 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[12/14 09:51:43 d2.data.build]: Using training sampler TrainingSampler
[12/14 09:51:43 d2.data.common]: Serializing 24 elements to byte tensors and concatenating them all ...
[12/14 09:51:43 d2.data.common]: Serialized dataset takes 3.17 MiB
WARNING [12/14 09:51:43 d2.solver.build]: SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. These values will be ignored.


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[12/14 09:51:43 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


...

[12/15 03:45:25 d2.utils.events]:  eta: 0:15:10  iter: 98339  total_loss: 0.5571  loss_cls: 0.03065  loss_box_reg: 0.1115  loss_mask: 0.1998  loss_rpn_cls: 0.05536  loss_rpn_loc: 0.1533  time: 0.6536  data_time: 0.0097  lr: 0.00025  max_mem: 8378M
[12/15 03:45:34 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 03:45:39 d2.utils.events]:  eta: 0:14:57  iter: 98359  total_loss: 0.5767  loss_cls: 0.03594  loss_box_reg: 0.1363  loss_mask: 0.1902  loss_rpn_cls: 0.05796  loss_rpn_loc: 0.1515  time: 0.6536  data_time: 0.0094  lr: 0.00025  max_mem: 8378M
[12/15 03:45:50 d2.utils.events]:  eta: 0:14:45  iter: 98379  total_loss: 0.5855  loss_cls: 0.03287  loss_box_reg: 0.1321  loss_mask: 0.2091  loss_rpn_cls: 0.04674  loss_rpn_loc: 0.1568  time: 0.6536  data_time: 0.0090  lr: 0.00025  max_mem: 8378M
[12/15 03:46:01 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM


[12/15 03:51:03 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 03:51:15 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 03:51:20 d2.utils.events]:  eta: 0:10:37  iter: 98879  total_loss: 0.5816  loss_cls: 0.02721  loss_box_reg: 0.1247  loss_mask: 0.1986  loss_rpn_cls: 0.05317  loss_rpn_loc: 0.1639  time: 0.6536  data_time: 0.0136  lr: 0.00025  max_mem: 8378M
[12/15 03:51:22 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 03:51:34 d2.utils.events]:  eta: 0:10:25  iter: 98899  total_loss: 0.5443  loss_cls: 0.0368  loss_box_reg: 0.1228  loss_mask: 0.2051  loss_rpn_cls: 0.04696  loss_rpn_loc: 0.1488  time: 0.6536  data_time: 0.0096  lr: 0.00025  max_mem: 8378M
[12/15 03:51:46 d2.utils.events]:  eta: 0:10:14  iter: 98919  total_loss: 0.5851  loss_cls: 0.03016  loss_box_reg: 0.1406

[12/15 03:56:49 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 03:56:53 d2.utils.events]:  eta: 0:05:52  iter: 99379  total_loss: 0.5762  loss_cls: 0.03241  loss_box_reg: 0.1261  loss_mask: 0.2027  loss_rpn_cls: 0.04684  loss_rpn_loc: 0.1504  time: 0.6537  data_time: 0.0094  lr: 0.00025  max_mem: 8378M
[12/15 03:57:04 d2.utils.events]:  eta: 0:05:41  iter: 99399  total_loss: 0.5354  loss_cls: 0.02996  loss_box_reg: 0.1295  loss_mask: 0.1934  loss_rpn_cls: 0.04672  loss_rpn_loc: 0.1472  time: 0.6537  data_time: 0.0089  lr: 0.00025  max_mem: 8378M
[12/15 03:57:15 d2.utils.events]:  eta: 0:05:29  iter: 99419  total_loss: 0.5684  loss_cls: 0.02364  loss_box_reg: 0.117  loss_mask: 0.2042  loss_rpn_cls: 0.05016  loss_rpn_loc: 0.1514  time: 0.6536  data_time: 0.0090  lr: 0.00025  max_mem: 8378M
[12/15 03:57:17 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[

[12/15 04:02:17 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 04:02:23 d2.utils.events]:  eta: 0:01:07  iter: 99879  total_loss: 0.5726  loss_cls: 0.0366  loss_box_reg: 0.1455  loss_mask: 0.1944  loss_rpn_cls: 0.05214  loss_rpn_loc: 0.1532  time: 0.6537  data_time: 0.0107  lr: 0.00025  max_mem: 8378M
[12/15 04:02:35 d2.utils.events]:  eta: 0:00:56  iter: 99899  total_loss: 0.5613  loss_cls: 0.0286  loss_box_reg: 0.128  loss_mask: 0.201  loss_rpn_cls: 0.04802  loss_rpn_loc: 0.1489  time: 0.6537  data_time: 0.0094  lr: 0.00025  max_mem: 8378M
[12/15 04:02:45 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f609b7cd200> to CPU due to CUDA OOM
[12/15 04:02:50 d2.utils.events]:  eta: 0:00:45  iter: 99919  total_loss: 0.5728  loss_cls: 0.03364  loss_box_reg: 0.1396  loss_mask: 0.2014  loss_rpn_cls: 0.04957  loss_rpn_loc: 0.1634  time: 0.6537  data_time: 0.0132  lr: 0.00025  max_mem: 8378M
[12/

AttributeError: 'NoneType' object has no attribute 'to'

# Register the testing coco instance

In [ ]:
register_coco_instances("test", {}, path_to_test_annotations,
                        path_to_test_images)


# Set the model's threshold、trained weight and get the test data

In [7]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
cfg.DATASETS.TEST = ("test", )
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,
                                 "model_final.pth")

# Predict one image and see the result

In [8]:
import cv2
predictor = DefaultPredictor(cfg)
img = cv2.imread("./dataset/test/TCGA-G9-6348-01Z-00-DX1.png")
output = predictor(img)
print(output)


{'instances': Instances(num_instances=100, image_height=1000, image_width=1000, fields=[pred_boxes: Boxes(tensor([[153.5361, 367.8488, 188.6047, 401.1132],
        [199.7998, 672.0380, 236.0706, 705.4072],
        [724.5292,  72.1132, 766.2985,  98.6640],
        [810.4023, 655.4791, 834.3146, 682.6049],
        [263.2572, 410.7906, 287.7430, 446.3961],
        [802.0037, 126.3244, 832.3513, 159.7401],
        [204.5383, 141.4646, 233.5141, 174.0149],
        [610.5238,   3.5143, 637.5432,  35.2490],
        [282.9544, 878.4156, 305.5258, 908.3585],
        [766.5189, 764.5666, 795.4493, 791.2693],
        [858.6918, 348.9867, 900.7940, 383.2085],
        [122.5992, 387.6637, 149.7551, 416.0755],
        [166.9514, 181.4566, 209.4774, 205.7096],
        [319.3407, 463.2062, 353.7127, 497.6846],
        [678.9890, 350.2056, 712.7902, 386.3116],
        [650.4493, 831.2872, 673.9141, 860.5164],
        [ 60.8645, 974.1350,  93.3481, 994.6210],
        [ 14.2592, 914.3790,  44.1129, 937.1

# Function to load the test image json file

In [9]:
def get_test_data(img_dir):
    TEST_SIZE = 6
    json_file_path = os.path.join(img_dir, "test_img_ids.json")
    with open(json_file_path) as f:
        info = json.load(f)
    filenames = []
    imgids = []
    for i in range(TEST_SIZE):
        filenames.append(info[i]['file_name'])
        imgids.append(info[i]['id'])
    for img, file in zip(imgids, filenames):
        yield img, file


# Output the predictions and save as json file

In [10]:
import json
import numpy as np
import pycocotools._mask as _mask
final_list = []

for id, file in get_test_data('./dataset/'):
    im = cv2.imread(f'./dataset/test/{file}')
    # load the image

    outputs = predictor(im)
    # outputs by the predictor
    field_list = outputs['instances'].get_fields()

    bbox_all = field_list['pred_boxes'].tensor
    score_all = field_list['scores']
    seg_all = field_list['pred_masks']
    # save the output data to list
    for ins in range(score_all.shape[0]):
        bbox_ins = bbox_all[ins].tolist()
        score_ins = score_all[ins].item()
        seg_ins = seg_all[ins].cpu()
        seg_ins = np.asfortranarray(seg_ins)
        h, w = seg_ins.shape
        rle_ins = _mask.encode(seg_ins.reshape((h, w, 1), order='F'))[0]
        rle_ins['counts'] = rle_ins['counts'].decode('ascii')
        final_list.append(dict(
            image_id=id,
            bbox=bbox_ins,
            score=score_ins,
            category_id=1,
            segmentation=rle_ins
        ))

print(final_list)

with open('./output/answer_100000iter.json', 'w') as f:
    json.dump(final_list, f)


[{'image_id': 1, 'bbox': [763.7533569335938, 646.3599853515625, 806.5863037109375, 693.4791259765625], 'score': 1.0, 'category_id': 1, 'segmentation': {'size': [1000, 1000], 'counts': 'kgZg0:kn07J4L4L3N3M2N2N1O2N2O0O1O100O100O1O10000O0100O10000O1O100O2N1O2O0O2N2N2M3N2M3M3M4K7Ghjk5'}}, {'image_id': 1, 'bbox': [724.7449340820312, 747.4542846679688, 769.1475830078125, 794.9697265625], 'score': 1.0, 'category_id': 1, 'segmentation': {'size': [1000, 1000], 'counts': '_gUf0`0dn05M4M1N3N2N3M101N1O2O0O100O2O0O1O10000O10O100000000O10000O2O0O2N2N2O1M3N3M3L4M6G_iR7'}}, {'image_id': 1, 'bbox': [258.77545166015625, 957.5438232421875, 287.44378662109375, 988.2833862304688], 'score': 0.9999998807907104, 'category_id': 1, 'segmentation': {'size': [1000, 1000], 'counts': 'Xlm7:mn01O1O1O100O001OO10000000001O1O0O2O1O1O1N2O1O00001O001OY[ge0'}}, {'image_id': 1, 'bbox': [332.3868408203125, 952.5560302734375, 351.0028076171875, 987.88525390625], 'score': 0.9999998807907104, 'category_id': 1, 'segmentation': 